# Data Preparation

This jupyter notebook comes from [this](https://www.kaggle.com/code/carlafgomes/fitbit-data-analysis-in-python#2-Data-Processing) original notebook Kaggle.  The original Kaggle notebook 
uses as input [this](https://www.kaggle.com/datasets/arashnic/fitbit) dataset at Kaggle. 

Before running this notebook,
you should run `jupyter_notebooks/heartrate_data_thinning.id.ipynb`.

The goal of this notebook is to produce, based on the 
Kaggle dataset, a file for each patient identified by his/her Id. This set 
of patient records is stored in the folder `patient_csv_records`



In [1]:
# this makes sure it starts looking for things from the CausalFitbit folder down.
import os
import sys
os.chdir('../')
sys.path.insert(0,os.getcwd())
print(os.getcwd())

C:\Users\rrtuc\Desktop\backed-up\python-projects\CausalFitbit


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import glob

In [3]:
fnames= ["dailyActivity",
"heartrate_reduced",
"sleepDay",
"weightLogInfo"]

def get_csv_path(fname):
 return f"input_data/Fitabase Data 4.12.16-5.12.16/{fname}_merged.csv"

dfs = [pd.read_csv(get_csv_path(fname)) for fname in fnames]

In [4]:
# number of unique Id's
for i in range(4):
    print(fnames[i], dfs[i].Id.nunique())

dailyActivity 33
heartrate_reduced 14
sleepDay 24
weightLogInfo 8


In [5]:
# column names
for i in range(4):
    print(f"*******{fnames[i]}\n", dfs[i].columns)

*******dailyActivity
 Index(['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance',
       'LoggedActivitiesDistance', 'VeryActiveDistance',
       'ModeratelyActiveDistance', 'LightActiveDistance',
       'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories'],
      dtype='object')
*******heartrate_reduced
 Index(['Id', 'Time', 'PulseHourlyAverage'], dtype='object')
*******sleepDay
 Index(['Id', 'SleepDay', 'TotalSleepRecords', 'TotalMinutesAsleep',
       'TotalTimeInBed'],
      dtype='object')
*******weightLogInfo
 Index(['Id', 'Date', 'WeightKg', 'WeightPounds', 'Fat', 'BMI',
       'IsManualReport', 'LogId'],
      dtype='object')


In [6]:
# check for missing values
for i in range(4):
    print(f"*******{fnames[i]}\n", dfs[i].isnull().sum())

*******dailyActivity
 Id                          0
ActivityDate                0
TotalSteps                  0
TotalDistance               0
TrackerDistance             0
LoggedActivitiesDistance    0
VeryActiveDistance          0
ModeratelyActiveDistance    0
LightActiveDistance         0
SedentaryActiveDistance     0
VeryActiveMinutes           0
FairlyActiveMinutes         0
LightlyActiveMinutes        0
SedentaryMinutes            0
Calories                    0
dtype: int64
*******heartrate_reduced
 Id                    0
Time                  0
PulseHourlyAverage    0
dtype: int64
*******sleepDay
 Id                    0
SleepDay              0
TotalSleepRecords     0
TotalMinutesAsleep    0
TotalTimeInBed        0
dtype: int64
*******weightLogInfo
 Id                 0
Date               0
WeightKg           0
WeightPounds       0
Fat               65
BMI                0
IsManualReport     0
LogId              0
dtype: int64


In [7]:
total_rows_fat = len(dfs[3]['Fat'])

print(f"Total number of rows in the 'Fat' column: {total_rows_fat}")

Total number of rows in the 'Fat' column: 67


In [8]:
# fnames= ["dailyActivity",
# "heartrate_seconds",
# "sleepDay",
# "weightLogInfo"]

# Since the 'Fat' columns has 67 rows, 
# and 65 of them are missing value, it will be dropped
dfs[3] = dfs[3].drop(columns=['Fat'])

# drop the columns that will not be needed
dfs[0] = dfs[0].drop(
    columns=['LoggedActivitiesDistance', 'TrackerDistance'])
dfs[2] = dfs[2].drop(columns=['TotalSleepRecords'])
dfs[3] = dfs[3].drop(columns=['IsManualReport', 'LogId', 'WeightKg'])

# rename ActiveDistance cols to something less stupid
dfs[0] = dfs[0].rename(columns={"SedentaryActiveDistance": "ActiveDistance0"})
dfs[0] = dfs[0].rename(columns={"LightActiveDistance": "ActiveDistance1"})
dfs[0] = dfs[0].rename(columns={"ModeratelyActiveDistance": "ActiveDistance2"})
dfs[0] = dfs[0].rename(columns={"VeryActiveDistance": "ActiveDistance3"})

# rename ActiveMinutes cols to something less stupid
dfs[0] = dfs[0].rename(columns={"SedentaryMinutes": "ActiveMinutes0"})
dfs[0] = dfs[0].rename(columns={"LightlyActiveMinutes": "ActiveMinutes1"})
dfs[0] = dfs[0].rename(columns={"FairlyActiveMinutes": "ActiveMinutes2"})
dfs[0] = dfs[0].rename(columns={"VeryActiveMinutes": "ActiveMinutes3"})

In [9]:
# dataframe shapes
for i in range(4):   
    print(f"*******{fnames[i]}\n", dfs[i].shape)

*******dailyActivity
 (940, 13)
*******heartrate_reduced
 (3332, 3)
*******sleepDay
 (413, 4)
*******weightLogInfo
 (67, 4)


In [10]:
# add new hour columns to sleepDay
dfs[2]['HoursAsleep'] = dfs[2]['TotalMinutesAsleep'] / 60
dfs[2]['HoursInBed'] = dfs[2]['TotalTimeInBed'] / 60
# remove minute columns
dfs[2] = dfs[2].drop(columns=['TotalMinutesAsleep'])
dfs[2] = dfs[2].drop(columns=['TotalTimeInBed'])

print(dfs[2].columns)

Index(['Id', 'SleepDay', 'HoursAsleep', 'HoursInBed'], dtype='object')


In [11]:
print(dfs[0].head())

           Id ActivityDate  TotalSteps  TotalDistance  ActiveDistance3  \
0  1503960366    4/12/2016       13162           8.50             1.88   
1  1503960366    4/13/2016       10735           6.97             1.57   
2  1503960366    4/14/2016       10460           6.74             2.44   
3  1503960366    4/15/2016        9762           6.28             2.14   
4  1503960366    4/16/2016       12669           8.16             2.71   

   ActiveDistance2  ActiveDistance1  ActiveDistance0  ActiveMinutes3  \
0             0.55             6.06              0.0              25   
1             0.69             4.71              0.0              21   
2             0.40             3.91              0.0              30   
3             1.26             2.83              0.0              29   
4             0.41             5.04              0.0              36   

   ActiveMinutes2  ActiveMinutes1  ActiveMinutes0  Calories  
0              13             328             728      1985 

In [12]:
# datatypes
for i in range(4):   
    print(f"*******{fnames[i]}\n", dfs[i].dtypes)

*******dailyActivity
 Id                   int64
ActivityDate        object
TotalSteps           int64
TotalDistance      float64
ActiveDistance3    float64
ActiveDistance2    float64
ActiveDistance1    float64
ActiveDistance0    float64
ActiveMinutes3       int64
ActiveMinutes2       int64
ActiveMinutes1       int64
ActiveMinutes0       int64
Calories             int64
dtype: object
*******heartrate_reduced
 Id                      int64
Time                   object
PulseHourlyAverage    float64
dtype: object
*******sleepDay
 Id               int64
SleepDay        object
HoursAsleep    float64
HoursInBed     float64
dtype: object
*******weightLogInfo
 Id                int64
Date             object
WeightPounds    float64
BMI             float64
dtype: object


In [13]:
# Transform all time column labels to DateTime data type
time_col_labels = ["ActivityDate", "Time", "SleepDay", "Date"]
for i in range(4): 
    str0 = time_col_labels[i]
    dfs[i][str0] = pd.to_datetime(dfs[i][str0])

In [14]:
# datatypes
for i in range(4):   
    print(f"*******{fnames[i]}\n", dfs[i].dtypes)

*******dailyActivity
 Id                          int64
ActivityDate       datetime64[ns]
TotalSteps                  int64
TotalDistance             float64
ActiveDistance3           float64
ActiveDistance2           float64
ActiveDistance1           float64
ActiveDistance0           float64
ActiveMinutes3              int64
ActiveMinutes2              int64
ActiveMinutes1              int64
ActiveMinutes0              int64
Calories                    int64
dtype: object
*******heartrate_reduced
 Id                             int64
Time                  datetime64[ns]
PulseHourlyAverage           float64
dtype: object
*******sleepDay
 Id                      int64
SleepDay       datetime64[ns]
HoursAsleep           float64
HoursInBed            float64
dtype: object
*******weightLogInfo
 Id                       int64
Date            datetime64[ns]
WeightPounds           float64
BMI                    float64
dtype: object


In [15]:
# check for duplicates. 
# df.duplicated() returns a dataframe
# of same shape as original df, but with bool entries.
# any() does OR on all bool entries.
for i in range(4):
    print(f"*******{fnames[i]}") 
    print(dfs[i].duplicated().any())     

*******dailyActivity
False
*******heartrate_reduced
False
*******sleepDay
True
*******weightLogInfo
False


In [16]:
# check for nulls = missing
# df.isnull() returns a dataframe
# of same shape as original df, but with bool entries.
# any() does OR on all bool entries.
for i in range(4):
    print(f"*******{fnames[i]}") 
    print(dfs[i].isnull().any())
          

*******dailyActivity
Id                 False
ActivityDate       False
TotalSteps         False
TotalDistance      False
ActiveDistance3    False
ActiveDistance2    False
ActiveDistance1    False
ActiveDistance0    False
ActiveMinutes3     False
ActiveMinutes2     False
ActiveMinutes1     False
ActiveMinutes0     False
Calories           False
dtype: bool
*******heartrate_reduced
Id                    False
Time                  False
PulseHourlyAverage    False
dtype: bool
*******sleepDay
Id             False
SleepDay       False
HoursAsleep    False
HoursInBed     False
dtype: bool
*******weightLogInfo
Id              False
Date            False
WeightPounds    False
BMI             False
dtype: bool


In [17]:
# renaming time columns all by same name
time_col_labels = ["ActivityDate", "Time", "SleepDay", "Date"]
for i in range(4): 
    str0 = time_col_labels[i]
    dfs[i] = dfs[i].rename(columns={str0: 'DateTime'})

In [18]:
# Concatenate the DataFrames vertically to combine all the data
combined_df = pd.concat(dfs, ignore_index=True)

# Sort the combined DataFrame by "DateTime" 
combined_df.sort_values(by='DateTime', inplace=True)

# Merge the data using forward fill (ffill)
combined_df.ffill(inplace=True)

print(combined_df.head())
print(combined_df.columns)

              Id   DateTime  TotalSteps  TotalDistance  ActiveDistance3  \
0     1503960366 2016-04-12     13162.0           8.50             1.88   
4611  7086361926 2016-04-12     13162.0           8.50             1.88   
536   5553957443 2016-04-12     11596.0           7.57             1.37   
505   4702921684 2016-04-12      7213.0           5.88             0.00   
474   4558609924 2016-04-12      5135.0           3.39             0.00   

      ActiveDistance2  ActiveDistance1  ActiveDistance0  ActiveMinutes3  \
0                0.55             6.06              0.0            25.0   
4611             0.55             6.06              0.0            25.0   
536              0.79             5.41              0.0            19.0   
505              0.00             5.85              0.0             0.0   
474              0.00             3.39              0.0             0.0   

      ActiveMinutes2  ActiveMinutes1  ActiveMinutes0  Calories  \
0               13.0           3

In [19]:
# add day of week column

combined_df['DayOfWeek'] = combined_df['DateTime'].dt.dayofweek

# Mapping the numeric representation of days to their respective names
combined_df['DayOfWeek'] = combined_df['DayOfWeek'].map({
    0: 'Mon',
    1: 'Tue',
    2: 'Wed',
    3: 'Thu',
    4: 'Fri',
    5: 'Sat',
    6: 'Sun'
})

In [20]:
print(combined_df.shape)

(4752, 19)


In [21]:
# round float64 columns
float_cols = combined_df.select_dtypes(include=['float64']).columns
combined_df[float_cols] = combined_df[float_cols].round(3)

print(combined_df.head())

              Id   DateTime  TotalSteps  TotalDistance  ActiveDistance3  \
0     1503960366 2016-04-12     13162.0           8.50             1.88   
4611  7086361926 2016-04-12     13162.0           8.50             1.88   
536   5553957443 2016-04-12     11596.0           7.57             1.37   
505   4702921684 2016-04-12      7213.0           5.88             0.00   
474   4558609924 2016-04-12      5135.0           3.39             0.00   

      ActiveDistance2  ActiveDistance1  ActiveDistance0  ActiveMinutes3  \
0                0.55             6.06              0.0            25.0   
4611             0.55             6.06              0.0            25.0   
536              0.79             5.41              0.0            19.0   
505              0.00             5.85              0.0             0.0   
474              0.00             3.39              0.0             0.0   

      ActiveMinutes2  ActiveMinutes1  ActiveMinutes0  Calories  \
0               13.0           3

In [22]:
# create patient files
def get_patient_csv_path(patient_id):
    return f"patient_csv_records/patient_{patient_id}.csv"
group_dict = combined_df.groupby('Id')
num = 0
patient_ids = []
for patient_id, patient_df in group_dict:
    patient_ids.append(patient_id)
    num += 1    
    patient_df.to_csv(get_patient_csv_path(patient_id), index=False)
    print(f"{num}, {patient_id}, {patient_df.shape}")

1, 1503960366, (58, 19)
2, 1624580081, (31, 19)
3, 1644430081, (34, 19)
4, 1844505072, (34, 19)
5, 1927972279, (37, 19)
6, 2022484408, (246, 19)
7, 2026352035, (64, 19)
8, 2320127002, (32, 19)
9, 2347167796, (274, 19)
10, 2873212765, (33, 19)
11, 3372868164, (20, 19)
12, 3977333714, (58, 19)
13, 4020332650, (119, 19)
14, 4057192912, (4, 19)
15, 4319703577, (59, 19)
16, 4388161847, (514, 19)
17, 4445114986, (59, 19)
18, 4558609924, (333, 19)
19, 4702921684, (59, 19)
20, 5553957443, (467, 19)
21, 5577150313, (434, 19)
22, 6117666160, (326, 19)
23, 6290855005, (29, 19)
24, 6775888955, (88, 19)
25, 6962181067, (513, 19)
26, 7007744171, (243, 19)
27, 7086361926, (55, 19)
28, 8053475328, (34, 19)
29, 8253242879, (19, 19)
30, 8378563200, (63, 19)
31, 8583815059, (31, 19)
32, 8792009665, (265, 19)
33, 8877689391, (117, 19)


In [23]:
# add velocity columns to each patient file

print_features = True
for patient_id in patient_ids:
    patient_df = pd.read_csv(get_patient_csv_path(patient_id))
    patient_df["DateTime"] = pd.to_datetime(patient_df["DateTime"])
    # add datetime diff column (in hours)
    patient_df["DateTimeDiff"] = patient_df["DateTime"].diff().dt.total_seconds() / 3600
    id_time_cols = ["Id", "DateTime", "DateTimeDiff", "DayOfWeek"]
    for col in patient_df.columns:
        if col not in id_time_cols:
            patient_df[f"{col}Vel"] = \
                (patient_df[col].diff()/patient_df["DateTimeDiff"]).round(3)     
    # change these 3 column names so they appear first in sorting
    patient_df =  patient_df.rename(columns={"Id": "1Id"})
    patient_df =  patient_df.rename(columns={"DateTime": "2DateTime"})
    patient_df =  patient_df.rename(columns={"DateTimeDiff": "3DateTimeDiff"})
    patient_df =  patient_df.rename(columns={"DayOfWeek": "4DayOfWeek"})
    # sort columns alphabetically
    patient_df = patient_df.reindex(sorted(patient_df.columns), axis=1)
    # restore old column names
    patient_df =  patient_df.rename(columns={"1Id": "Id"})
    patient_df =  patient_df.rename(columns={"2DateTime": "DateTime"})
    patient_df =  patient_df.rename(columns={"3DateTimeDiff": "DateTimeDiff"}) 
    patient_df =  patient_df.rename(columns={"4DayOfWeek": "DayOfWeek"})
    
    if print_features:
        id_time_cols = ['Id', 'DateTime', 'DateTimeDiff', "DayOfWeek"]
        features = [col for col in patient_df.columns if col not in id_time_cols]
        print(features)
        print()
        print(patient_df.columns)
        print_features = False
    
    patient_df.to_csv(get_patient_csv_path(patient_id), index=False)  

['ActiveDistance0', 'ActiveDistance0Vel', 'ActiveDistance1', 'ActiveDistance1Vel', 'ActiveDistance2', 'ActiveDistance2Vel', 'ActiveDistance3', 'ActiveDistance3Vel', 'ActiveMinutes0', 'ActiveMinutes0Vel', 'ActiveMinutes1', 'ActiveMinutes1Vel', 'ActiveMinutes2', 'ActiveMinutes2Vel', 'ActiveMinutes3', 'ActiveMinutes3Vel', 'BMI', 'BMIVel', 'Calories', 'CaloriesVel', 'HoursAsleep', 'HoursAsleepVel', 'HoursInBed', 'HoursInBedVel', 'PulseHourlyAverage', 'PulseHourlyAverageVel', 'TotalDistance', 'TotalDistanceVel', 'TotalSteps', 'TotalStepsVel', 'WeightPounds', 'WeightPoundsVel']

Index(['Id', 'DateTime', 'DateTimeDiff', 'DayOfWeek', 'ActiveDistance0',
       'ActiveDistance0Vel', 'ActiveDistance1', 'ActiveDistance1Vel',
       'ActiveDistance2', 'ActiveDistance2Vel', 'ActiveDistance3',
       'ActiveDistance3Vel', 'ActiveMinutes0', 'ActiveMinutes0Vel',
       'ActiveMinutes1', 'ActiveMinutes1Vel', 'ActiveMinutes2',
       'ActiveMinutes2Vel', 'ActiveMinutes3', 'ActiveMinutes3Vel', 'BMI',
    